In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import pipeline

In [ ]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

classifier = pipeline("sentiment-analysis",model= model_name)

result = classifier('He is not good boy')
print(result)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


[{'label': '1 star', 'score': 0.4733272194862366}]


In [ ]:
result = classifier('This is a best movie')
print(result)

[{'label': '5 stars', 'score': 0.8458419442176819}]


## tokenizer, encoding, decoding

In [ ]:
token = AutoTokenizer.from_pretrained(model_name)

classifier = pipeline("sentiment-analysis",model= model_name,tokenizer=token)

result = classifier('He is not good boy')
print(result)

Device set to use cuda:0


[{'label': '1 star', 'score': 0.4733272194862366}]


In [ ]:
tokenizer = token.tokenize('He is not good boy')
print(tokenizer)

ids = token.convert_tokens_to_ids(tokenizer)
print(ids)

['he', 'is', 'not', 'good', 'boy']
[10191, 10127, 10497, 12050, 14140]


In [ ]:
encoded_input = token('He is a good boy')
print(encoded_input)

decoded_input = token.decode(encoded_input['input_ids'])
print(decoded_input)

{'input_ids': [101, 10191, 10127, 143, 12050, 14140, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
[CLS] he is a good boy [SEP]


# Work with Custom data

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset('imdb')


In [ ]:
dataset.shape

{'train': (25000, 2), 'test': (25000, 2), 'unsupervised': (50000, 2)}

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
from datasets import DatasetDict

# Reduce each split to 1500 samples (randomly shuffled)
dataset = DatasetDict({
    "train": dataset["train"].shuffle(seed=42).select(range(1500)),
    "test": dataset["test"].shuffle(seed=42).select(range(1500)),
    "unsupervised": dataset["unsupervised"].shuffle(seed=42).select(range(1500))
})


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1500
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 1500
    })
})

## Pre-processing

In [ ]:
train_data = dataset['train']
test_data = dataset['test']

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenizer_fun(dataset):
  return tokenizer(dataset['text'],padding = "max_length",truncation=True )


tokenized_data = dataset.map(tokenizer_fun, batched = True)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1500
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1500
    })
})

In [ ]:
tokenized_data['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

# Set the training Argument

In [ ]:
!pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 76.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:
      Successfully uninstalled transformers-4.56.2


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch', # Corrected argument name
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01
)

print(training_args)

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=True,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_use_gather_object=False,

# Initialized the model

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data['train'],
    eval_dataset = tokenized_data['test']
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.243957


TrainOutput(global_step=94, training_loss=0.32619671111411236, metrics={'train_runtime': 204.3696, 'train_samples_per_second': 7.34, 'train_steps_per_second': 0.46, 'total_flos': 394666583040000.0, 'train_loss': 0.32619671111411236, 'epoch': 1.0})

# Save Model

In [ ]:
model.save_pretrained('./fine-tuned-model')